# Django Intro

"Django is a high-level Python Web framework that encourages rapid development and clean, pragmatic design. Built by experienced developers, it takes care of much of the hassle of Web development, so you can focus on writing your app without needing to reinvent the wheel. It’s free and open source."

--Excerpt from the djangoproject website

## Getting started

You will need to install a few modules to get started with django:

- django
- rest-framework
- pillow (python image package)
- django-filter

** We may add packages down the line...but this should be a good starting point **

Before you install the packages remember to create a conda environment that we will use for development. We will call this environment: **"django-polls"**

To add the above packages you will need to PIP install them:

`pip install django`

### Alternative install

I've gone ahead and created a conda environment that should be easier to work with. Download it from slack.

## Topics to cover this week

- Django models and databases
- HTTP requests
- Django forms
- Templates in django 

### Starting a Django Project

A django project is made up of a set of files(or directories)which communicate with each other. A common (basic) file structure for a django project may look like this for example:

    mysite/
    manage.py
    mysite/
        __init__.py
        settings.py
        urls.py
        wsgi.py
        
To get started with our example app (Polls) we will start by using the command line django tool that should have come installed with your environment.

#### This is the command that you will place inside of your command prompt/terminal
        django-admin startproject polls
        
Our file directory should now have the following structure:

    polls/
    manage.py
    polls/
        __init__.py
        settings.py
        urls.py
        wsgi.py # The meaning of this acronym is Web Server Gateway Interface
        
So now, when we can test our newly created app by going back to the command prompt,changing our directory to `polls` and running:

    python manage.py runserver
    
While the server is running, go to the browser and navigate to the localhost URL:

    localhost:8000
    

    

### Migrations

GREAT!!! We have a working server now.

You may have seen a few warnings when running the server for the first time. This is showing up because we have built in django models that come with every newly created app. Those models include:

    auth, contenttypes, and  sessions.
    
Let's create our first migration shall we...?

To do this we will use the command prompt again, utilizing the Django command tool `manage.py`

    python manage.py migrate
    
    
When this runs successfully, you will see the following output:

    Operations to perform:
      Apply all migrations: admin, auth, contenttypes, sessions
      Running migrations:
      Applying contenttypes.0001_initial... OK
      Applying auth.0001_initial... OK
      Applying admin.0001_initial... OK
      Applying admin.0002_logentry_remove_auto_add... OK
      Applying contenttypes.0002_remove_content_type_name... OK
      Applying auth.0002_alter_permission_name_max_length... OK
      Applying auth.0003_alter_user_email_max_length... OK
      Applying auth.0004_alter_user_username_opts... OK
      Applying auth.0005_alter_user_last_login_null... OK
      Applying auth.0006_require_contenttypes_0002... OK
      Applying auth.0007_alter_validators_add_error_messages... OK
      Applying auth.0008_alter_user_username_max_length... OK
      Applying sessions.0001_initial... OK
      
Now, you may have noticed that this ran successfully...but we never connected to a PostGreSQL database. So how is this possible?

This happens because by default Django ships with a connection to SQLite and creates a database for your when you do a first migration.

But we want to connect to PostGreSQL as this is where we will do our development. Here's how we do that:

In `settings.py` we need to navigate to the database portion of the file and add these lines:
           
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'polls',
        'PASSWORD': 'Nina3689',
        'USER': 'postgres',
        
 **Make sure that your PGAdmin is running and the Database `polls` has been created**

### Example of pre-created django model

In [3]:
'''
from __future__ import unicode_literals

import json

from django.conf import settings
from django.contrib.admin.utils import quote
from django.contrib.contenttypes.models import ContentType
from django.db import models
from django.urls import NoReverseMatch, reverse
from django.utils import timezone
from django.utils.encoding import force_text, python_2_unicode_compatible
from django.utils.text import get_text_list
from django.utils.translation import ugettext, ugettext_lazy as _

ADDITION = 1
CHANGE = 2
DELETION = 3


class LogEntryManager(models.Manager):
    use_in_migrations = True

    def log_action(self, user_id, content_type_id, object_id, object_repr, action_flag, change_message=''):
        if isinstance(change_message, list):
            change_message = json.dumps(change_message)
        return self.model.objects.create(
            user_id=user_id,
            content_type_id=content_type_id,
            object_id=force_text(object_id),
            object_repr=object_repr[:200],
            action_flag=action_flag,
            change_message=change_message,
        )


@python_2_unicode_compatible
class LogEntry(models.Model):
    action_time = models.DateTimeField(
        _('action time'),
        default=timezone.now,
        editable=False,
    )
    user = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        models.CASCADE,
        verbose_name=_('user'),
    )
    content_type = models.ForeignKey(
        ContentType,
        models.SET_NULL,
        verbose_name=_('content type'),
        blank=True, null=True,
    )
    object_id = models.TextField(_('object id'), blank=True, null=True)
    # Translators: 'repr' means representation (https://docs.python.org/3/library/functions.html#repr)
    object_repr = models.CharField(_('object repr'), max_length=200)
    action_flag = models.PositiveSmallIntegerField(_('action flag'))
    # change_message is either a string or a JSON structure
    change_message = models.TextField(_('change message'), blank=True)

    objects = LogEntryManager()

    class Meta:
        verbose_name = _('log entry')
        verbose_name_plural = _('log entries')
        db_table = 'django_admin_log'
        ordering = ('-action_time',)

    def __repr__(self):
        return force_text(self.action_time)

    def __str__(self):
        if self.is_addition():
            return ugettext('Added "%(object)s".') % {'object': self.object_repr}
        elif self.is_change():
            return ugettext('Changed "%(object)s" - %(changes)s') % {
                'object': self.object_repr,
                'changes': self.get_change_message(),
            }
        elif self.is_deletion():
            return ugettext('Deleted "%(object)s."') % {'object': self.object_repr}

        return ugettext('LogEntry Object')

    def is_addition(self):
        return self.action_flag == ADDITION

    def is_change(self):
        return self.action_flag == CHANGE

    def is_deletion(self):
        return self.action_flag == DELETION

    def get_change_message(self):
        """
        If self.change_message is a JSON structure, interpret it as a change
        string, properly translated.
        """
        if self.change_message and self.change_message[0] == '[':
            try:
                change_message = json.loads(self.change_message)
            except ValueError:
                return self.change_message
            messages = []
            for sub_message in change_message:
                if 'added' in sub_message:
                    if sub_message['added']:
                        sub_message['added']['name'] = ugettext(sub_message['added']['name'])
                        messages.append(ugettext('Added {name} "{object}".').format(**sub_message['added']))
                    else:
                        messages.append(ugettext('Added.'))

                elif 'changed' in sub_message:
                    sub_message['changed']['fields'] = get_text_list(
                        sub_message['changed']['fields'], ugettext('and')
                    )
                    if 'name' in sub_message['changed']:
                        sub_message['changed']['name'] = ugettext(sub_message['changed']['name'])
                        messages.append(ugettext('Changed {fields} for {name} "{object}".').format(
                            **sub_message['changed']
                        ))
                    else:
                        messages.append(ugettext('Changed {fields}.').format(**sub_message['changed']))

                elif 'deleted' in sub_message:
                    sub_message['deleted']['name'] = ugettext(sub_message['deleted']['name'])
                    messages.append(ugettext('Deleted {name} "{object}".').format(**sub_message['deleted']))

            change_message = ' '.join(msg[0].upper() + msg[1:] for msg in messages)
            return change_message or ugettext('No fields changed.')
        else:
            return self.change_message

    def get_edited_object(self):
        "Returns the edited object represented by this log entry"
        return self.content_type.get_object_for_this_type(pk=self.object_id)

    def get_admin_url(self):
        """
        Returns the admin URL to edit the object represented by this log entry.
        """
        if self.content_type and self.object_id:
            url_name = 'admin:%s_%s_change' % (self.content_type.app_label, self.content_type.model)
            try:
                return reverse(url_name, args=(quote(self.object_id),))
            except NoReverseMatch:
                pass
        return None
'''

'\nfrom __future__ import unicode_literals\n\nimport json\n\nfrom django.conf import settings\nfrom django.contrib.admin.utils import quote\nfrom django.contrib.contenttypes.models import ContentType\nfrom django.db import models\nfrom django.urls import NoReverseMatch, reverse\nfrom django.utils import timezone\nfrom django.utils.encoding import force_text, python_2_unicode_compatible\nfrom django.utils.text import get_text_list\nfrom django.utils.translation import ugettext, ugettext_lazy as _\n\nADDITION = 1\nCHANGE = 2\nDELETION = 3\n\n\nclass LogEntryManager(models.Manager):\n    use_in_migrations = True\n\n    def log_action(self, user_id, content_type_id, object_id, object_repr, action_flag, change_message=\'\'):\n        if isinstance(change_message, list):\n            change_message = json.dumps(change_message)\n        return self.model.objects.create(\n            user_id=user_id,\n            content_type_id=content_type_id,\n            object_id=force_text(object_id),\n 

Creating a new module for model testing can be done with the following command:

    python manage.py startapp model
    
Once the module has been created add the module to the `INSTALLED_APPS` dictionary located in the `settings.py` file

    '''
    INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'models',
    ]
    '''
    
Inside of the `models` module:

    polls/
    models/
        models.py #write your code in this file
    manage.py
    
create your model class:
   
   
    class Customer(models.Model):
    first_name = models.TextField(max_length=100)
    last_name = models.TextField(max_length=100)
    
    
    class Product(models.Model):
        name = models.CharField(max_length=100)
        price = models.DecimalField(max_digits=6, decimal_places=2)
        
Once your models have been created (and your file saved), you should then be able to head to the command prompt and enter the following commands:
    
    - python manage.py makemigrations
    - python manage.py migrate
    
If the above commands worked successfully the output for each command should be (each in respect to the order of the commands given above):
    
        Migrations for 'models':
          models\migrations\0001_initial.py
            - Create model Customer
            - Create model Product
            
            
            Operations to perform:
              Apply all migrations: admin, auth, contenttypes, models, sessions
            Running migrations:
              Applying models.0001_initial... OK
              
              
Now, head back to PGAdmin and you should now see the tables your created inside of your database.

## Model Homework

   Inside of the `model.py` we've been working in, create the following models(classes)
       
       - Address
       - Store_location
       - Merchant
           -merchant_id      
       -Inventory
       -Payment
       -Rating
       -Marketing
       
       
  Inside of each of these models, create at least 3 fields(I would encourage you to play around with adding more though)